In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
import ast



In [391]:
data = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/Coding with Computed LBP Features V6.csv'
df = pd.read_csv(data)

print(type(df['Computed LBP'][0]))

df['Computed LBP'] = df['Computed LBP'].apply(lambda x: list(map(float, x.split(','))) if pd.notnull(x) else x)
df['Computed LBP'] = df['Computed LBP'].apply(lambda x: np.array(x).flatten() if isinstance(x, list) else x)
print(type(df['Computed LBP'][0]))


<class 'str'>
<class 'numpy.ndarray'>


In [392]:
df.shape

(149, 8)

In [393]:
df.head()

,Subject,Filename,OnsetFrame,ApexFrame,OffsetFrame,Action Units,Estimated Emotion,Computed LBP
0,1,EP02_01f,46,59,86,12,happiness,"[2.0, 4.0, 5.0, 5.0, 3.0, 1.0, 0.0, 1.0, 9.0, ..."
1,1,EP19_05f,396,416,446,4+L10,disgust,"[1.0, 1.0, 1.0, 3.0, 4.0, 5.0, 3.0, 3.0, 1.0, ..."
2,1,EP19_06f,36,71,161,4+5+L10,disgust,"[2.0, 4.0, 4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 2.0, ..."
3,2,EP01_11f,46,91,96,15,repression,"[3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 2.0, ..."
4,2,EP02_04f,31,79,141,12+15,repression,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ..."


In [ ]:
col_names = df.columns

col_names

In [394]:

# X = df.drop(['Subject', 'Filename', 'OnsetFrame', 'ApexFrame', 'OffsetFrame',
#        'Action Units', 'Estimated Emotion'], axis=1)
# y = df['Estimated Emotion']

# X = np.array(X)

X = df['Computed LBP'].to_list()
X = np.array(X)

y = df['Estimated Emotion']


print(df.at[0, 'Computed LBP'])
# print(df['Computed LBP'])
print(df.at[0, 'Estimated Emotion'])
print(X.shape)



[2. 4. 5. 5. 3. 1. 0. 1. 9. 4. 5. 3. 3. 4. 5. 2. 5. 5. 5. 3. 3. 3. 3. 3.
 3. 2. 0. 3. 4. 5. 4. 3. 2. 3. 5. 5. 2. 2. 2. 1. 5. 4. 3. 2. 3. 5. 3. 2.
 5. 9. 1. 2. 2. 5. 4. 4. 2. 0. 2. 3. 9. 1. 3. 3. 5. 0. 2. 3. 5. 3. 3. 4.
 5. 4. 4. 4. 4. 5. 4. 4. 5. 4. 3. 2. 3. 5. 4. 5. 3. 5.]
happiness
(149, 90)


In [395]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# df['Computed LBP'] = df['Computed LBP'].apply(convert_to_array)
# X_test['ApexFrame'] = X_test['ApexFrame'].str.strip("/")
# X_train['ApexFrame'] = X_train['ApexFrame'].str.strip("/")
# X_train['ApexFrame'] = np.where(X_train['ApexFrame'] == '', np.nan, X_train['ApexFrame'])

# Convert 'ApexFrame' to float, replacing NaN values with 0 (or any other desired value)
# df['Computed LBP'] = df['Computed LBP'].apply(lambda x: ' '.join(x))
# df['Computed LBP'] = df['Computed LBP'].apply(lambda x: ' '.join(x))
# X_train['Computed LBP'] = pd.to_numeric(X_train['Computed LBP'])
# X_test['Computed LBP'] = pd.to_numeric(X_test['Computed LBP'])
# X_train.flatten()


In [ ]:
from pandas import unique


print(X_test['Computed LBP'].unique())
print(X_train['Computed LBP'].unique())


In [ ]:
X_train.shape, X_test.shape
y_train.shape, y_test.shape

In [398]:
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
import joblib


# Classifier initialization
svm_classifier = SVC(kernel='linear', C=1.0)

# Training the Classifier
svm_classifier.fit(X_train, y_train)
joblib.dump(svm_classifier, '/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Model_ApexFrameV6_37percent.joblib')


['/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Model_ApexFrameV6_37percent.joblib']

In [400]:
trained_SVM_Model = joblib.load('/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Model_ApexFrameV6.joblib')

# Test prediction using Test sample 
predictions = trained_SVM_Model.predict(X_test)

print(predictions)

accuracy = accuracy_score(y_test, predictions)
print (f"Accuracy: {accuracy * 100: .2f}%")

['disgust' 'disgust' 'disgust' 'surprise' 'surprise' 'disgust' 'happiness'
 'repression' 'happiness' 'surprise' 'surprise' 'disgust' 'disgust'
 'happiness' 'disgust' 'disgust' 'disgust' 'disgust' 'happiness'
 'repression' 'disgust' 'happiness' 'happiness' 'disgust' 'disgust'
 'repression' 'happiness' 'surprise' 'repression' 'happiness' 'disgust'
 'surprise' 'disgust' 'disgust' 'disgust' 'disgust' 'repression' 'disgust'
 'happiness' 'happiness' 'repression' 'disgust' 'repression' 'disgust'
 'disgust']
Accuracy:  37.78%


In [410]:
import cv2
from skimage import feature
import numpy as np
from sklearn.svm import SVC
import joblib

def compute_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = feature.local_binary_pattern(gray, P=8, R=1, method="uniform")

    lbp_features_flat = lbp.flatten()[:90]

    return lbp_features_flat

def classifyME(imagePath, svmModelPath):
    image = cv2.imread(imagePath)

    lbpfeatures = compute_lbp(image)

    if len(lbpfeatures) != 90:
        print(f"Error: Expected 90 features, but got {len(lbpfeatures)} features.")
        return None

    lbpfeatures = np.array([lbpfeatures])

    trained_SVM_Model = joblib.load(svmModelPath)

    prediction = trained_SVM_Model.predict(lbpfeatures)

    return prediction


imagePathtoClassify = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/Extracted_ApexFrames/EP06_07_frame225.jpg'
svm_model_path = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Model_ApexFrameV6_37percent.joblib'

predictedME = classifyME(imagePathtoClassify, svm_model_path)

print(f"Micro-Expression Present is: {predictedME}")


Micro-Expression Present is: ['happiness']
